# Long format generator
### this script converts the current Yarrowia media pretest comparing 96vs24 manual vs machine pipetting in xlsx into long format in csv.

#### this can be repeated by making some minor changes to be applied to create the same files for Green Values instead of OD.

In [11]:
import pandas as pd

pathxlsx = 'data/Growth Profiler Results 96v24 pretest.xlsx'

# Define mappings for all sheets:
mappings = {
    'OD96_Manual': pd.DataFrame({
        'WellID': ['A1','A2','A3','B1','B2','B3','C1','C2','C3', 
            'A4','A5','A6','B4','B5','B6','C4','C5','C6',
            'A7','A8','A9','B7','B8','B9','C7','C8','C9',
            'A10','A11','A12','B10','B11','B12','C10','C11','C12'],
        'Media': ['Media_1']*9 + ['Media_2']*9 + ['Media_3']*9 + ['Media_4']*9,
        'Replicate': [1,2,3,4,5,6,7,8,9,
                    1,2,3,4,5,6,7,8,9,
                    1,2,3,4,5,6,7,8,9,
                    1,2,3,4,5,6,7,8,9],
        'Condition': ['OD96_Manual']*36
    }),
    'OD96_Robot': pd.DataFrame({
        'WellID': ['A1','A2','A3','A4','A5','A6','D4','D5','D6','A7','A8','A9','A10','A11','A12'],
        'Media': ['Media_1']*3 + ['Media_2']*6 + ['Media_3']*3 + ['Media_4']*3,
        'Replicate': [1,2,3,1,2,3,4,5,6,1,2,3,1,2,3],
        'Condition': ['OD96_Robot']*15
    }),
    'OD24_Manual': pd.DataFrame({
        'WellID': ['A1','A2','A3','A4','A5','A6','B1','B2','B3','B4','B5','B6'],
        'Media': ['Media_1']*3 + ['Media_2']*3 + ['Media_3']*3 + ['Media_4']*3,
        'Replicate': [1,2,3,1,2,3,1,2,3,1,2,3],
        'Condition': ['OD24_Manual']*12
    }),
    'OD24_Robot': pd.DataFrame({
        'WellID': ['A1','A2','A3','A4','A5','A6','D4','D5','D6','B1','B2','B3','B4','B5','B6'],
        'Media': ['Media_1']*3 + ['Media_2']*6 + ['Media_3']*3 + ['Media_4']*3,
        'Replicate': [1,2,3,1,2,3,4,5,6,1,2,3,1,2,3],
        'Condition': ['OD24_Robot']*15
    })
}

def load_and_tidy(sheet_name):
    # Load sheet
    df = pd.read_excel(pathxlsx, sheet_name=sheet_name, header=0)

    # Melt to long format
    long_df = df.melt(id_vars='Time (min)', var_name='WellID', value_name='OD600')

    # Convert time to hours
    long_df['Time'] = long_df['Time (min)'] / 60
    long_df = long_df.drop(columns=['Time (min)'])

    # Filter mapping for wells present in this sheet
    mapping_filtered = mappings[sheet_name][mappings[sheet_name]['WellID'].isin(long_df['WellID'].unique())]

    # Merge
    long_df = long_df.merge(mapping_filtered, on='WellID')

    # Sort and reset index
    long_df = long_df.sort_values(by=['Condition','Media','Replicate','Time']).reset_index(drop=True)

    return long_df

# Load all sheets and combine
all_long_dfs = []
for sheet in mappings.keys():
    tidy_df = load_and_tidy(sheet)
    all_long_dfs.append(tidy_df)

full_long_df = pd.concat(all_long_dfs, ignore_index=True)

print(full_long_df.head())


  WellID  OD600      Time    Media  Replicate    Condition
0     A1  0.185  0.000000  Media_1          1  OD96_Manual
1     A1  0.179  0.087500  Media_1          1  OD96_Manual
2     A1  0.179  0.820833  Media_1          1  OD96_Manual
3     A1  0.185  1.325000  Media_1          1  OD96_Manual
4     A1  0.185  1.820833  Media_1          1  OD96_Manual


In [12]:
full_long_df.to_csv('data/GrowthProfilerResults_Long_Pretest.csv', index=False)